# Data Mining Project 3 - Link Analysis
*Date : 2018/12/25*

    Hist and PageRank 
    (Lecture 7, P37, random jumping probability, i.e., damping factor=0.15) and calculate authority, hub and PageRank values for the following 8 graphs 
    - 6 graphs in project3dataset  
    - 1 graphs from project1 transaction data (connect items in each row, bidirected or directed) 
    - SimRank to calculate pair-wise similarity of nodes (choice any parameter C you like) , using 
    - first 5 graphs of project3dataset. 

    Find a way (e.g., add/delete some links) to increase hub, authority, and PageRank of Node 1 in first 3 graphs respectively.

    You should write a report for your system, including: 
    - Implementation detail 
    - Result analysis and discussion 
    - Computation performance analysis 
    - Discussion (what you learned from this project and your comments about this project) 

    1. More limitations about link analysis algorithms 
    2. Can link analysis algorithms really find the “important” pages from Web? 
    3. What are practical issues when implement these algorithms in a real Web? 
        - Performance discussion (time cost) 
    4. What do the result say for your actor/movie graph?  
    5. Any new idea about the link analysis algorithm? 
    6. What is the effect of “C” parameter in SimRank? 
    7. Design a new link-based similarity measurement 
## Dataset
- project3dataset
- transaction data

## Algorithm:
### HITS 
```
 1 G := set of pages
 2 for each page p in G do
 3   p.auth = 1 // p.auth is the authority score of the page p
 4   p.hub = 1 // p.hub is the hub score of the page p
 5 function HubsAndAuthorities(G)
 6   for step from 1 to k do // run the algorithm for k steps
 7     norm = 0
 8     for each page p in G do  // update all authority values first
 9       p.auth = 0
10       for each page q in p.incomingNeighbors do // p.incomingNeighbors is the set of pages that link to p
11          p.auth += q.hub
12       norm += square(p.auth) // calculate the sum of the squared auth values to normalise
13     norm = sqrt(norm)
14     for each page p in G do  // update the auth scores 
15       p.auth = p.auth / norm  // normalise the auth values
16     norm = 0
17     for each page p in G do  // then update all hub values
18       p.hub = 0
19       for each page r in p.outgoingNeighbors do // p.outgoingNeighbors is the set of pages that p links to
20         p.hub += r.auth
21       norm += square(p.hub) // calculate the sum of the squared hub values to normalise
22     norm = sqrt(norm)
23     for each page p in G do  // then update all hub values
24       p.hub = p.hub / norm   // normalise the hub values
```
- *reference from [wiki](https://en.wikipedia.org/wiki/HITS_algorithm)*
### PageRank

>![](https://wikimedia.org/api/rest_v1/media/math/render/svg/7a88d72ae8cfe827f8872d60bbe5905a8165b587)

>![](https://wikimedia.org/api/rest_v1/media/math/render/svg/6bb0f1469218a064274fd4691143e9ce64639dc2)

*reference from [wiki](https://en.wikipedia.org/wiki/PageRank)*
### SimRank

>![](https://wikimedia.org/api/rest_v1/media/math/render/svg/da632465bb3b520d1fce5d79f0c4627eb16766c4)

>![](https://wikimedia.org/api/rest_v1/media/math/render/svg/dafa90b2a86f89371a5951c779ba07c7d1ab6e97)

*reference from [wiki](https://en.wikipedia.org/wiki/SimRank)*
## Code
---

In [2]:
import numpy as np
import pandas as pd
import time

In [3]:
class page():
    """define the page class"""
    def __init__(self, name):
        self.hub = 1       
        self.auth = 1
        self.pr = 1
        self.name = name     
        self.parents = []  # point out
        self.childs = []   # point in

In [4]:
def getGraph(filename):
    """read the text file and return the graph in dictionary form"""
    graph = dict()
    with open(filename) as f:
        for line in f:
            #read a line from file and split by comma
            # p : parent(in) ; c : child(out)
            p, c = [int(item) for item in line.rstrip().split(',')]
            if p not in graph.keys():
                graph.update({p:page(p)})
            if c not in graph.keys():
                graph.update({c:page(c)})
            graph[p].childs.append(graph[c])
            graph[c].parents.append(graph[p]) 
    return graph

In [5]:
def AHPS(graph, simrank=True):
    """calculate hub , auth, PageRank and SimRank"""
    # Calculate auth.
    norm = 0
    for name in graph:
        p = graph[name]
        p.auth = 0
        for parent in p.parents:
            p.auth += parent.hub
        norm += np.power(p.auth, 2)
    norm = np.sqrt(norm)
    for name in graph:
        p = graph[name]
        p.auth /= norm

    # Calculate hub
    norm = 0
    for name in graph:
        p = graph[name]
        p.hub = 0
        for child in p.childs:
            p.hub += child.auth
        norm += np.power(p.hub, 2)
    norm = np.sqrt(norm)
    for name in graph:
        p = graph[name]
        p.hub /= norm

    # Calculate PageRank 
    d = 0.15
    e = 0.01
    N = len(graph)
    # k=0 initial
    for name in graph:
        graph[name].pr = 1/N
    flag = True
    k = 1
    norm_temp = 1
    while(flag):
        count = 0
        norm = 0
        pr_temp = []
        for name in graph:
            P_i = graph[name]
            pr_temp = P_i.pr*norm_temp
            P_i.pr = d/N + (1-d)*sum([P_j.pr/len(P_j.childs) for P_j in P_i.parents])
            norm += P_i.pr
            if abs(pr_temp-P_i.pr) < 3:
                count += 1
        norm_temp = norm
        for name in graph:
            P_i = graph[name]
            P_i.pr /= norm
        if count == N:
            flag = False
        else:
            k +=1
    
    # Calculate SimRank
    if simrank:
        start = time.time()
        C = 0.6  # 0 < C < 1
        # k=0 (initial)
        S = np.zeros(N*N).reshape(N, N)
        for i in range(N):
            S[i, i] = 1
        k = 1
        flag = True
        while(flag):
            S_temp = S.copy()
            for a in graph:
                for b in graph:
                    I_a = list(graph.keys()).index(graph[a].name)
                    I_b = list(graph.keys()).index(graph[b].name)
                    temp = len(graph[a].parents)*len(graph[b].parents)
                    if I_a == I_b:
                        S[I_a, I_b] = 1
                    elif temp == 0:
                        S[I_a, I_b] = 0
                    else:
                        count = 0
                        for i in graph[a].parents:
                            for j in graph[b].parents:
                                I_i = list(graph.keys()).index(i.name)
                                I_j = list(graph.keys()).index(j.name)
                                count += S[I_i, I_j]
                        S[I_a, I_b] = C/temp*count
            if (S_temp == S).all():
                flag = False
            else:
                k += 1
        end = time.time()
        print('[Calculate SimRank] time: {}'.format(end-start))
        
    # print result
    df = pd.DataFrame(index=graph.keys(),columns=['Auth', 'Hub', 'PageRank'])
    for p in graph:
        df.loc[graph[p].name, :] = [round(item, 6) for item in [graph[p].auth, graph[p].hub, graph[p].pr]]
    print(df)
    if simrank:
        print('SimRank:\n{}'.format(S))

## Result
---

### Graph1
- Dataset: hw3dataset/graph_1.txt
- Graph: 6 nodes, 5 edges
- time: <15ms

In [6]:
%%time
# Read file and initial graph
graph = getGraph('hw3dataset/graph_1.txt')
AHPS(graph)

[Calculate SimRank] time: 0.0
       Auth       Hub  PageRank
1         0  0.447214  0.060716
2  0.447214  0.447214  0.112325
3  0.447214  0.447214  0.156192
4  0.447214  0.447214  0.193479
5  0.447214  0.447214  0.225174
6  0.447214         0  0.252114
SimRank:
[[1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 1.]]
Wall time: 12 ms


#### Increase Hubness 
- Node 1 **link to all nodes in graph**
- hub(node1): 0.447214 -> 0.913812

In [7]:
graph = getGraph('hw3dataset/graph_1.txt')
for name in graph:
    if name == 1:
        continue
    if graph[name] not in graph[1].childs:
        graph[1].childs.append(graph[name])
        graph[name].parents.append(graph[1])
AHPS(graph, simrank=False)

       Auth       Hub  PageRank
1         0  0.913812  0.064885
2  0.242536  0.203069  0.075916
3  0.485071  0.203069  0.140445
4  0.485071  0.203069  0.195294
5  0.485071  0.203069  0.241916
6  0.485071         0  0.281544


#### Increase Authority  
- Node1 **receives all node cittaions in graph.**
- auth(node1): 0 -> 0.912871

In [8]:
graph = getGraph('hw3dataset/graph_1.txt')
for name in graph:
    if name == 1:
        continue
    if graph[name] not in graph[1].parents:
        graph[1].parents.append(graph[name])
        graph[name].childs.append(graph[1])
AHPS(graph, simrank=False)

       Auth       Hub  PageRank
1  0.912871  0.076696   0.34838
2  0.182574  0.460179  0.315477
3  0.182574  0.460179  0.153432
4  0.182574  0.460179  0.084563
5  0.182574  0.460179  0.055294
6  0.182574  0.383482  0.042854


#### Increase PageRank  
- The same way as increasing authority
- Node1 **receives all node cittaions in graph.**
- PageRank(node1): 0.060716 -> 0.34838


### Graph2
- Dataset: /hw3dataset/graph_2.txt
- Graph: 5 nodes, 5 edges (a circle)
- time: <15ms


In [9]:
%%time
# Read file and initial graph
graph = getGraph('hw3dataset/graph_2.txt')
AHPS(graph)

[Calculate SimRank] time: 0.0
       Auth       Hub PageRank
1  0.447214  0.447214      0.2
2  0.447214  0.447214      0.2
3  0.447214  0.447214      0.2
4  0.447214  0.447214      0.2
5  0.447214  0.447214      0.2
SimRank:
[[1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1.]]
Wall time: 14 ms


#### Increase Hubness 
- Node 1 **link to all nodes in graph**
- hub(node1): 0.447214 -> 0.889

In [10]:
graph = getGraph('hw3dataset/graph_2.txt')
for name in graph:
    if name == 1:
        continue
    if graph[name] not in graph[1].childs:
        graph[1].childs.append(graph[name])
        graph[name].parents.append(graph[1])
AHPS(graph, simrank=False)

       Auth       Hub  PageRank
1  0.267261  0.889001  0.242671
2  0.267261     0.254  0.087968
3  0.534522     0.254  0.162741
4  0.534522     0.254  0.226298
5  0.534522     0.127  0.280322


#### Increase Authority  
- Node1 **receives all node cittaions in graph.**
- auth(node1): 0.447214 -> 0.894427

In [19]:
graph = getGraph('hw3dataset/graph_2.txt')
for name in graph:
    if name == 1:
        continue
    if graph[name] not in graph[1].parents:
        graph[1].parents.append(graph[name])
        graph[name].childs.append(graph[1])
AHPS(graph, simrank=False)

       Auth       Hub  PageRank
1  0.894427  0.104257  0.356288
2  0.223607  0.521286  0.326337
3  0.223607  0.521286  0.162185
4  0.223607  0.521286   0.09242
5  0.223607  0.417029   0.06277


#### Increase PageRank  
- The same way as increasing authority
- Node1 **receives all node cittaions in graph.**
- PageRank(node1): 0.2 -> 0.35288


### Graph3
- Dataset: /hw3dataset/graph_3.txt
- Graph: 4 nodes, 6 edges
- time: <15ms

In [20]:
%%time
# Read file and initial graph
graph = getGraph('hw3dataset/graph_3.txt')
AHPS(graph)

[Calculate SimRank] time: 0.0009970664978027344
       Auth       Hub  PageRank
1  0.316228  0.392232  0.149042
2  0.632456  0.588348  0.275727
3  0.632456  0.588348  0.376387
4  0.316228  0.392232  0.198845
SimRank:
[[1.         0.         0.42857143 0.        ]
 [0.         1.         0.         0.42857143]
 [0.42857143 0.         1.         0.        ]
 [0.         0.42857143 0.         1.        ]]
Wall time: 9.97 ms


#### Increase Hubness 
- Node 1 **link to all nodes in graph**
- hub(node1): 0.392232 -> 0.737865

In [21]:
graph = getGraph('hw3dataset/graph_3.txt')
for name in graph:
    if name == 1:
        continue
    if graph[name] not in graph[1].childs:
        graph[1].childs.append(graph[name])
        graph[name].parents.append(graph[1])
AHPS(graph, simrank=False)

       Auth       Hub  PageRank
1  0.235702  0.737865   0.15416
2  0.471405  0.421637  0.197839
3  0.707107  0.421637  0.395865
4  0.471405  0.316228  0.252137


#### Increase Authority  
- Node1 **receives all node cittaions in graph.**
- auth(node1): 0.316228 -> 0.707107

In [22]:
graph = getGraph('hw3dataset/graph_3.txt')
for name in graph:
    if name == 1:
        continue
    if graph[name] not in graph[1].parents:
        graph[1].parents.append(graph[name])
        graph[name].childs.append(graph[1])
AHPS(graph, simrank=False)

       Auth       Hub  PageRank
1  0.707107  0.210819  0.283502
2  0.471405  0.527046  0.336704
3  0.471405  0.632456  0.270123
4  0.235702  0.527046  0.109671


#### Increase PageRank  
- The same way as increasing authority
- Node1 **receives all node cittaions in graph.**
- PageRank(node1): 0.0149 -> 0.283502


### Graph4
- Dataset: /hw3dataset/graph_4.txt
- Graph: 7 nodes, 18 edges (the example in Lecture3, p29)
- time: <30ms

In [23]:
%%time
# Read file and initial graph
graph = getGraph('hw3dataset/graph_4.txt')
AHPS(graph)

[Calculate SimRank] time: 0.022925138473510742
       Auth       Hub  PageRank
1  0.534522  0.573405  0.256862
2  0.400892  0.176432  0.150563
3  0.400892  0.308757  0.124098
4  0.267261  0.441081  0.088812
5  0.534522  0.441081  0.246298
6  0.133631  0.352865  0.071019
7  0.133631  0.176432  0.062347
SimRank:
[[1.         0.16616792 0.15766797 0.16537597 0.1478556  0.22608541
  0.10466654]
 [0.16616792 1.         0.21707712 0.18287626 0.214697   0.10114374
  0.26460879]
 [0.15766797 0.21707712 1.         0.26201824 0.19876792 0.26139016
  0.26264631]
 [0.16537597 0.18287626 0.26201824 1.         0.15909518 0.34435668
  0.34435668]
 [0.1478556  0.214697   0.19876792 0.15909518 1.         0.09377117
  0.22441919]
 [0.22608541 0.10114374 0.26139016 0.34435668 0.09377117 1.
  0.08871336]
 [0.10466654 0.26460879 0.26264631 0.34435668 0.22441919 0.08871336
  1.        ]]
Wall time: 32.9 ms


### Graph5
- Dataset: /hw3dataset/graph_5.txt
- Graph:  469 nodes, 1102 edges
- time: **356s**

In [24]:
%%time
# Read file and initial graph
graph = getGraph('hw3dataset/graph_5.txt')
AHPS(graph)

[Calculate SimRank] time: 762.4361662864685
         Auth       Hub  PageRank
1           0  0.012684  0.001076
2           0  0.002985  0.001076
3           0  0.011192  0.001076
4           0  0.044768  0.001076
5           0  0.033576  0.001076
6    0.009498  0.011938  0.001229
7    0.009498  0.053722  0.001168
8    0.009498         0  0.001207
9    0.009498         0  0.001305
10   0.009498         0  0.001305
11   0.009498  0.008954  0.001207
12   0.009498  0.050738  0.001168
13   0.009498  0.001492  0.001305
14   0.009498         0  0.001305
15   0.009498         0  0.001168
16   0.009498         0   0.00133
17   0.009498         0   0.00132
18   0.028495         0  0.002421
19   0.009498         0  0.002093
20   0.009498         0  0.001233
21   0.028495         0  0.004888
22   0.028495         0  0.004549
23   0.009498         0  0.001233
24   0.037994         0   0.00565
25   0.018997         0  0.001884
26   0.009498         0  0.001225
27   0.009498  0.014177  0.001947
28  

### Graph6
- Dataset: /hw3dataset/graph_6.txt
- Graph: 1228 nodes, 5220 edges
- time: 750ms

In [25]:
%%time
# Read file and initial graph
graph = getGraph('hw3dataset/graph_6.txt')
AHPS(graph, simrank=False)

          Auth       Hub  PageRank
1            0  0.068027  0.000514
2      0.00607         0  0.000717
3     0.003035         0    0.0006
4     0.003035         0  0.000611
5     0.003035         0  0.000605
6     0.033386         0  0.001681
7      0.00607   0.07534  0.001027
8      0.00607  0.110462  0.000732
9     0.015175  0.090186  0.000939
10    0.003035         0  0.000585
11    0.009105         0  0.001783
12    0.003035         0  0.000611
13    0.009105  0.024596  0.000763
14    0.003035         0    0.0006
15    0.048561         0    0.0026
16    0.003035         0  0.000646
17    0.009105         0  0.000798
18    0.003035         0   0.00058
19    0.003035         0  0.000605
20    0.003035         0  0.000635
21     0.00607         0  0.000734
22    0.015175         0  0.000802
23    0.003035         0  0.000562
24    0.003035         0   0.00058
25     0.01821  0.081544  0.001139
26    0.003035         0  0.000653
27    0.003035         0  0.000583
28    0.003035      

### Graph7
- from Project1 IBM transation data
- Dataset: /hw3dataset/graph_7.txt
- Graph: 15622 nodes, 29572 edges
- time: 51.6s

In [29]:
"""convert original data and save as graph_7.txt"""
with open('hw3dataset/data.txt', 'r') as f:
    with open('hw3dataset/graph_7.txt', 'w') as f_out:
        out = list()
        for line in f:
            v1, v2, v3 = line.split()
            f_out.write('{},{}\n'.format(v2,v3))

In [30]:
%%time
# Read file and initial graph
graph = getGraph('hw3dataset/graph_7.txt')
AHPS(graph, simrank=False)

           Auth       Hub  PageRank
1             0  0.021913   5.1e-05
2             0  0.006527   5.1e-05
3             0  0.014919   5.1e-05
4             0  0.031703   5.1e-05
5             0  0.008392   5.1e-05
6             0  0.010723   5.1e-05
7             0  0.018649   5.1e-05
8             0  0.009791   5.1e-05
9             0  0.027041   5.1e-05
10            0  0.013987   5.1e-05
11            0   0.04569   5.1e-05
12            0  0.004196   5.1e-05
13            0  0.008392   5.1e-05
14            0  0.025176   5.1e-05
15            0  0.021446   5.1e-05
16            0  0.011656   5.1e-05
17            0  0.010723   5.1e-05
18            0  0.018649   5.1e-05
19            0  0.013521   5.1e-05
20            0  0.013054   5.1e-05
21            0  0.004662   5.1e-05
22            0  0.017717   5.1e-05
23            0  0.015385   5.1e-05
24            0  0.024244   5.1e-05
25     0.009339  0.013521  0.000173
26            0  0.005595   5.1e-05
27            0  0.028906   

## Dicussion
---

### Computation performance analysis
$N$: # of pages, $M$: # of edges, $k$: # of iteration
- Hub and Auth: $O(N)$
- PageRank: $O(k*(N+M))$
    - 設置較低的門檻值(e)，可以減少迭帶次數(k)。
- SimRank: $O(k*N^2)$
    - Graph5: N = 469，因此大幅增加計算時間
    - S(a,b) = S(b,a) 此為對稱關係，若避免重複求值，可以減少一半計算量。

### Other
- 無法以單一指表來衡量page重要性
- 在真實情況下，page數相當龐大，許多演算法難以實作
- 如何設計一個適切的資料結構來計算page重要性相當重要，可以由加速存取資料的速度來改善效能

## Q & A
---

### Q1. More limitations about link analysis algorithms
- 難以用單一指標來衡量Page
- page數快速增加，造成演算法實作的困難

### Q2. Can link analysis algorithms really find the “important” pages from Web?
以PageRank為例
- 如果引用許多高PageRank的網頁，便可以增加自身的PageRank
- 因此當演算法被解析，反常的page大量出現，就難以衡量page的重要性。

### Q3. What are practical issues when implement these algorithms in a real Web? (Performance discussion)

以PageRank為例
- Initial 
    - time cost: $O(N)$
    - 需要知道**page的總數**來初始化，然而page數量不斷增加，難以計算總數。
- Update: 
    - time cost: $O(N^2)$
    - 要更新PageRank直到穩定的階段，不但時間複雜度提高，再到穩定階段前可能又有新的page出現。
- 因此 **page數量不斷上升** 以及 **page的關係不斷改變** ，成為難以應用於現實情況的關鍵。
    

### Q4. What do the result say for your actor/movie graph?  
*pass*

### Q5. Any new idea about the link analysis algorithm? 
*pass*

### Q6. What is the effect of “C” parameter in SimRank? 
假設page x 指向 page a, page b
- 因為S(x, x)=1，不該使 S(c, d) = S(x, x) = 1，所以乘上 **C** 使 S(c, d) !=1

### Q7. Design a new link-based similarity measurement 
*pass*